<a href="https://colab.research.google.com/github/nikunjchapadia/fastbook/blob/master/nlp_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# language model 
#  word - categorical variable 
 
#  tokenization 
# word based, subword based, character based 

In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 49.4 MB/s 
     |████████████████████████████████| 189 kB 51.6 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 558 kB 57.4 MB/s 
     |████████████████████████████████| 51 kB 269 kB/s 
     |████████████████████████████████| 130 kB 56.2 MB/s 
Mounted at /content/gdrive


In [3]:
from fastbook import *
from IPython.display import display,HTML

In [4]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [5]:
path

Path('/root/.fastai/data/imdb')

In [7]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [10]:
files
files[0]

Path('/root/.fastai/data/imdb/unsup/44142_0.txt')

In [11]:
txt = files[0].open().read(); txt[:75]

'Let me set the stage. I love well-crafted art-house dramas and romances fro'

In [12]:
txt

'Let me set the stage. I love well-crafted art-house dramas and romances from East Asia. Christmas In August (1998), One Fine Spring Day (2001), Spring Summer Fall Winter and Spring (2003), Asako In Ruby Shoes (2000), Rules of Dating (2005), Art Museum By the Zoo (1998), and most of Yasujiro Ozu\'s films were very enjoyable to me. Add another 50 or so similar films to that recommendation list, and one could be forgiven to conclude that I not only like deliberately-paced, slow-burning art-house cinema, but I\'m actually a big fan of the genre.<br /><br />To date I\'ve seen four of Sang-soo Hong\'s films and one thing is certain: the artistic merits of Virgin Stripped Bare By Her Bachelors (2000) was a fluke of sorts within Hong\'s otherwise unimpressive portfolio that treads dangerously close to mirroring something you\'d see on Cinemax Late Night. A half dozen sex scenes? Check. Complete absence of character development? Check. Dull-as-dirt filler material? Check. In all honesty, there

In [15]:
# Tokenizer - spacy
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#1166) ['Let','me','set','the','stage','.','I','love','well','-','crafted','art','-','house','dramas','and','romances','from','East','Asia','.','Christmas','In','August','(','1998',')',',','One','Fine'...]


In [17]:
first(spacy(['The U.S. dollar $1 is $1.00']))

(#8) ['The','U.S.','dollar','$','1','is','$','1.00']

In [22]:
# lowercase everything 
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#1315) ['xxbos','xxmaj','let','me','set','the','stage','.','i','love','well','-','crafted','art','-','house','dramas','and','romances','from','xxmaj','east','xxmaj','asia','.','xxmaj','christmas','xxmaj','in','xxmaj','august'...]


In [23]:
coll_repr(tkn('&copy; Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [26]:
# subword tokenizatiom 
# grab 2000 movie reviews
txts = L(o.open().read() for o in files[:2000])
txts

(#2000) ['Let me set the stage. I love well-crafted art-house dramas and romances from East Asia. Christmas In August (1998), One Fine Spring Day (2001), Spring Summer Fall Winter and Spring (2003), Asako In Ruby Shoes (2000), Rules of Dating (2005), Art Museum By the Zoo (1998), and most of Yasujiro Ozu\'s films were very enjoyable to me. Add another 50 or so similar films to that recommendation list, and one could be forgiven to conclude that I not only like deliberately-paced, slow-burning art-house cinema, but I\'m actually a big fan of the genre.<br /><br />To date I\'ve seen four of Sang-soo Hong\'s films and one thing is certain: the artistic merits of Virgin Stripped Bare By Her Bachelors (2000) was a fluke of sorts within Hong\'s otherwise unimpressive portfolio that treads dangerously close to mirroring something you\'d see on Cinemax Late Night. A half dozen sex scenes? Check. Complete absence of character development? Check. Dull-as-dirt filler material? Check. In all hones

In [33]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return ' '.join(first(sp([txt]))[:40])

In [34]:
subword(1000)

'▁Le t ▁me ▁set ▁the ▁st age . ▁I ▁love ▁well - c ra f ted ▁art - ho use ▁drama s ▁and ▁r om ance s ▁from ▁E as t ▁A s ia . ▁Chris t ma s ▁In'

In [35]:
subword(200)

'▁ L e t ▁ m e ▁ s e t ▁the ▁st a g e . ▁I ▁ l o ve ▁w e ll - c ra f t ed ▁ ar t - h o us e ▁d'

In [36]:
subword(10000)

'▁Let ▁me ▁set ▁the ▁stage . ▁I ▁love ▁well - craft ed ▁art - house ▁dramas ▁and ▁romance s ▁from ▁East ▁As ia . ▁Christmas ▁In ▁August ▁(199 8) , ▁One ▁Fine ▁S p ring ▁Day ▁(200 1 ), ▁S'

In [37]:
# numerization 
toks = tkn(txt)

(#1315) ['xxbos','xxmaj','let','me','set','the','stage','.','i','love'...]

In [39]:
toks200 = txts[:200].map(tkn)
toks200

(#200) [['xxbos', 'xxmaj', 'let', 'me', 'set', 'the', 'stage', '.', 'i', 'love', 'well', '-', 'crafted', 'art', '-', 'house', 'dramas', 'and', 'romances', 'from', 'xxmaj', 'east', 'xxmaj', 'asia', '.', 'xxmaj', 'christmas', 'xxmaj', 'in', 'xxmaj', 'august', '(', '1998', ')', ',', 'xxmaj', 'one', 'xxmaj', 'fine', 'xxmaj', 'spring', 'xxmaj', 'day', '(', '2001', ')', ',', 'xxmaj', 'spring', 'xxmaj', 'summer', 'xxmaj', 'fall', 'xxmaj', 'winter', 'and', 'xxmaj', 'spring', '(', '2003', ')', ',', 'xxmaj', 'asako', 'xxmaj', 'in', 'xxmaj', 'ruby', 'xxmaj', 'shoes', '(', '2', 'xxrep', '3', '0', ')', ',', 'xxmaj', 'rules', 'of', 'xxmaj', 'dating', '(', '2005', ')', ',', 'xxmaj', 'art', 'xxmaj', 'museum', 'xxmaj', 'by', 'the', 'xxmaj', 'zoo', '(', '1998', ')', ',', 'and', 'most', 'of', 'xxmaj', 'yasujiro', 'xxmaj', 'ozu', "'s", 'films', 'were', 'very', 'enjoyable', 'to', 'me', '.', 'xxmaj', 'add', 'another', '50', 'or', 'so', 'similar', 'films', 'to', 'that', 'recommendation', 'list', ',', 'and', 

In [40]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2152) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','a','and','of','to','is','in','it','i'...]"

In [42]:
nums = num(toks)[:20]
nums

TensorText([   0,   84,  237,    9,  874,   11,    0,  144,   86,   24,    0,  387,   24,  201, 1260,   13,    0,   50,    0,    0])

In [43]:
' '.join(num.vocab[o] for o in nums)

'xxunk me set the stage . xxunk love well - xxunk art - house dramas and xxunk from xxunk xxunk'

In [ ]:
# create batches
# 90 tokens 6 batch each wiith 15 length

# w1 w2 w3 w4 w5 w6 w7 w8 w9 w10 w11 w12 w13 w14 w15
# w16 w17 w18 w19 w20 w21 w22 w23 w24 w25 w26 w27 w28 w29 w30 
# w31 w32 w33 w34 w35 w36 w37 w38 w39 w40 w41 w42 w43 w44 w45
# w46 w47 w48 w49 w50 w51 w52 w53 w54 w55 w56 w57 w58 w59 w60
# w61 w62 w63 w64 w65 w66 w67 w68 w69 w70 w71 w72 w73 w74 w75
# w76 w77 w78 w79 w80 w81 w82 w83 w84 w85 w86 w87 w88 w89 w90 

# First 
# w1 w2 w3 w4 w5
# w16 w17 w18 w19 w20
# w31 w32 w33 w34 w35
# w46 w47 w48 w49 w50
# w61 w62 w63 w64 w65
# w76 w77 w78 w79 w80

# Then second
# w6 w7 w8 w9 w10
# w21 w22 w23 w24 w25
# w36 w37 w38 w39 w40
# w51 w52 w53 w54 w55
# w66 w67 w68 w69 w70
# w81 w82 w83 w84 w85

# Last and final 
# w11 w12 w13 w14 w15
# w26 w27 w28 w29 w30 
# w41 w42 w43 w44 w45
# w56 w57 w58 w59 w60
# w71 w72 w73 w74 w75
# w86 w87 w88 w89 w90 

In [44]:
nums200 = toks200.map(num)
nums200

(#200) [TensorText([  2,   8, 295,  ...,   0,   0,  11]),TensorText([   2,   12,    7,  416,    7,  297,   37,    0,    8,    0,   24,    8,  637,    0,   36,   11,   26,   12,  155,  697, 1282,   27,    9,  334,    8, 1591,    8, 1592,   17,    9,  489,   11,
          19,   38,   12,  256,    0,   27,   66,  120,   13,   21,   16,  535,   44,   14,  116,   11,   19,  112,  265,   18,   24,   40,  181,  121,   18,  417,   61,   24,   41,    8, 1593, 1283,
         895,   17,   80, 1284,  344,   17,  446,  896,   11,   19,  221,    9,  297,   17,    9,  418,  289,   37, 1591,    8, 1592,   36,   25,    9,    0,  297,   19,  159,  265,   13,    9,   33,
         248, 1594,   27,   84,   11,    8,  364,   41,   10, 1595,  897,   15,   63,  249,   17,   21,   33,   10,   74,   83,  345,   12,  226,  322,   13,    0,   15,   46,   17,    9,    0,   13,
          94,  197, 1596,    8,    0,    0,   17,    7,    9,    7,    0,   37,  260,    5,  142,  525,   36,   10,   45,   20,   23,  195,

In [45]:
dl = LMDataLoader(nums200)
dl

In [46]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [ ]:
# offset by 1 x and y 

In [48]:
# independent variable
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj let me set the stage . i love well - xxunk art - house dramas and xxunk from'

In [49]:
# dependent variable
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj let me set the stage . i love well - xxunk art - house dramas and xxunk from xxmaj'

In [51]:
# Training a text classifier 
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

db = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb,
    splitter=RandomSplitter(0.1)
    )
dls_lm = db.dataloaders(path, path=path, bs=128, seq_ln=80)

In [52]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this is a brilliant film . xxmaj the story starts off in xxmaj romania , where xxmaj billy xxmaj crystal stars as an agent who can talk anybody into doing anything . xxmaj there is a really funny scene here , where xxmaj crystal is a child , and talks his father , who is a rabbi into eating pork ( you xxunk may know that this is a parody","xxmaj this is a brilliant film . xxmaj the story starts off in xxmaj romania , where xxmaj billy xxmaj crystal stars as an agent who can talk anybody into doing anything . xxmaj there is a really funny scene here , where xxmaj crystal is a child , and talks his father , who is a rabbi into eating pork ( you xxunk may know that this is a parody of"
1,"very closely to find it wanting . xxmaj i 'm willing to let all kinds of things go for the sake of suspension of disbelief , but this movie just pushes it too far . xxmaj there 's a scene where xxmaj charlie and his buddy decide to sell videotapes of students being beaten up , and the whole school goes crazy trying to buy them . i just did n't get","closely to find it wanting . xxmaj i 'm willing to let all kinds of things go for the sake of suspension of disbelief , but this movie just pushes it too far . xxmaj there 's a scene where xxmaj charlie and his buddy decide to sell videotapes of students being beaten up , and the whole school goes crazy trying to buy them . i just did n't get it"


In [61]:
dls_lm.show

<bound method Datasets.show of (#90000) [(TensorText([    2,    13,    32,    21, 16424,   101,    18,    15,   128,  3713,   872,    10,   107,    14,    45,  1542,   187,    46,     9,    98,   850,    11,    91,    15,   897,    28,    55,
           39,    75,  1262, 16249,    23,    10,     8,     9,  1127,    12,   202,    18,    20,    32,    93,    38,   187,    15,  2342,    56,  1054,   200,    15,  2385,   201,    10,     8,
           81,   254,    43,    13,  2932,    14,    21,    31,    69,    17,   294,   204,    15,    17,   115,    60,   278, 12832,    11,   278, 30848,    12,  4693,    21,  7541,   278, 11442,
           15,  1137,    10,     8,     9,  4961,    87,    13,  2650,   329,  1120,    15,    20,    31,    17,    25,    45,   113,    73, 14361,    12,   677,  6908,   174,    10,     8,    46,
          243,    22,     9,     8,   250,     8,  3965,    22,   791,    15,   214,    18,    15,  2611,  1176,    12,   924,    51,    13,  3810,   244,    14,   67

In [62]:
# Fine tuning Language model  
# predict the next word of movei review 
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('first_epoch')

In [ ]:
learn.load('first_epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
# save model except last layer 
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

In [ ]:
# classifier dataloaders
db = DataBlock(
    # from folder will take care padding 
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y= parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
)

dls_clas = db.dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
# for classifier 
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
# add padding to make every token to same size 

In [ ]:
learn = text_classifier_learner(
    dls_clas, 
    AWD_LSTM,
    drop_mult=0.5,
    metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
# fine tunning the classifier
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-2/(2.6**4),5e-2))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))